In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7498, 1)  # default port was 7497

Error 321, reqId -1: Error validating request.-'ca' : cause - The API interface is currently in Read-Only mode.
completed orders request timed out


<IB connected to 127.0.0.1:7498 clientId=1>

In [11]:
import pandas as pd
import time
from datetime import datetime, timedelta, timezone
import numpy as np

In [3]:
contract1 = Stock('SH', 'SMART', 'USD')
ib.reqMktData(contract1, '', False, False)
t1 = ib.ticker(contract1)

Error 354, reqId 3: Requested market data is not subscribed.Delayed market data is available.Error&BEST/STK/Top&BEST/STK/Top, contract: Stock(symbol='SH', exchange='SMART', currency='USD')


In [4]:
ib.reqMarketDataType( 3 )

In [21]:
def read_two_tick_prices(stock1, stock2, minutes):
    contract1 = Stock(stock1, 'SMART', 'USD')
    ib.reqMktData(contract1, '', False, False)
    
    contract2 = Stock(stock2, 'SMART', 'USD')
    ib.reqMktData(contract2, '', False, False)

    t1 = ib.ticker(contract1)
    t2 = ib.ticker(contract2)
    ib.sleep(0.06)
    
    data = []
    pre_row_data = None
    
    t_start = time.time()
    t_end = t_start + 60 * minutes
    print(datetime.now())
    
    while time.time() < t_end:
        ib.sleep(0)
        row_data = [t1.bid, t1.bidSize,t1.ask, t1.askSize, t2.bid, t2.bidSize,  t2.ask, t2.askSize]
        if row_data != pre_row_data:
            row_time = t1.time.replace(tzinfo=timezone.utc).astimezone(tz=None)
            row = [row_time] + row_data
            data.append(row)
            pre_row_data = row_data
        else:
            pass
    
    return pd.DataFrame(data, columns=['time', 'bid1', 'bid_size1', 'ask1', 'ask_size1', 'bid2', 'bid_size2', 'ask2', 'ask_size2'])

In [24]:
%time
df = read_two_tick_prices('SH', 'SDS', 300)
df.to_csv('SH_SDS_data.csv')

Wall time: 0 ns
2021-02-19 11:26:40.629531


In [26]:
ib.disconnect()